In [ ]:
from PIL import Image
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
import os
DRIVE = '/gdrive'
# WRK_DIR = DRIVE + '/MyDrive/Colab Notebooks/(Step 2) Normalized Images -> Unsupervised Features -> Conventional Classification'

drive.mount('/gdrive', force_remount=True)
# os.chdir(WRK_DIR)

Mounted at /gdrive


In [ ]:
from keras.utils import to_categorical

Local_images_array = np.load('/gdrive/My Drive/local_images_512.npy')

b_label = [0] * 800
n_label = [1] * 800
inv_label = [2] * 800
ins_label = [3] * 800
y = b_label + n_label + inv_label + ins_label
labels = to_categorical(y)

In [ ]:
import numpy as np

def split_data(X, y, test_size=0.2, random_state=5432):
    # Reshape X and y so that every 8 samples are treated as one datum
    n_datapoints = X.shape[0] // 8
    print(n_datapoints)
    X = X[:n_datapoints*8].reshape(n_datapoints, 8, X.shape[1], X.shape[2], X.shape[3])
    # X = X[:n_datapoints*8].reshape(n_datapoints, 8, X.shape[1])
    y = y[:n_datapoints*8].reshape(n_datapoints, 8, y.shape[1])
    
    # Split the data into training and testing sets
    n_test = int(n_datapoints * test_size)
    np.random.seed(random_state)
    indices = np.random.permutation(n_datapoints)
    X_train = X[indices[n_test:], :]
    y_train = y[indices[n_test:], :, :]
    X_test = X[indices[:n_test], :]
    y_test = y[indices[:n_test], :, :]
    
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = split_data(Local_images_array, labels)
X_train = X_train.reshape((320*8,512, 512, 3))
y_train = y_train.reshape((320*8,4))
X_test = X_test.reshape((80*8,512, 512, 3))
y_test = y_test.reshape((80*8,4))

400


In [ ]:
from keras.models import load_model

#Loading Model
model = load_model("/gdrive/My Drive/Colab Notebooks/Mahdi_Inceptionv3_Model_Local_1.h5")

In [ ]:
'''
Feature Extraction
'''
from keras.models import Model

feature_model = Model(inputs=model.input, outputs=model.layers[-2].output)
features_train = feature_model.predict(X_train)
features_test = feature_model.predict(X_test)

20/20 [==============================] - 151s 8s/step


In [ ]:
print(features_train.shape)
print(features_test.shape)

(2560, 512)
(640, 512)


In [ ]:
'''
Batch Prediction Voting
'''

def batch_voting(clf_y_preds):
  y_pred = []
  for i in range(len(y_test)):
    start = 8*i
    end = 8*i +8
    # get the labels of the current group
    group_pred = clf_y_preds[start:end]
    # use voting to predict the label of the group
    pred_label = np.argmax(np.bincount(group_pred))
    y_pred.append(pred_label)
  
  return y_pred

In [ ]:
test = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 0, 0],
                [0, 0, 0, 0]])

array([4])

In [ ]:
import numpy as np

train_labels = np.array([])
for pred in y_train:
  train_labels = np.append(train_labels, np.argmax(pred))

test_labels = np.array([])
for pred in y_test:
  test_labels = np.append(test_labels, np.argmax(pred))

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

SVM_model = SVC()
SVM_model.fit(features_train, train_labels)
SVM_y_preds = SVM_model.predict(features_test)

KNN_model = KNeighborsClassifier(n_neighbors=11)
KNN_model.fit(features_train, y_train)
KNN_y_preds = KNN_model.predict(features_test)
KNN_y_preds_voting = []
for i in range(len(y_test)):
  start = 8*i
  end = 8*i +8
  # get the labels of the current group
  group_pred = KNN_y_preds[start:end]
  class_preds = {
      0: 0,
      1: 0,
      2: 0,
      3: 0
  }
  # print(group_pred)
  for pred in group_pred:
    print(pred, np.where(pred == 1))
  # use voting to predict the label of the group
  # print(np.bincount(group_pred[0].astype(np.float)))
  pred_label = np.argmax(np.bincount(group_pred[0]))
  KNN_y_preds_voting.append(pred_label)

XGB_model = XGBClassifier()
XGB_model.fit(features_train, y_train)
XGB_y_preds = XGB_model.predict(features_test)

[0. 1. 0. 0.] (array([1]),)
[0. 0. 0. 0.] (array([], dtype=int64),)
[0. 0. 1. 0.] (array([2]),)
[0. 0. 0. 0.] (array([], dtype=int64),)
[0. 0. 1. 0.] (array([2]),)
[0. 0. 1. 0.] (array([2]),)
[0. 0. 0. 0.] (array([], dtype=int64),)
[0. 0. 0. 0.] (array([], dtype=int64),)


TypeError: ignored

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_labels, SVM_y_preds))
print(classification_report(y_test, KNN_y_preds_voting))
print(classification_report(y_test, XGB_y_preds))

              precision    recall  f1-score   support

           0       0.75      0.58      0.66       200
           1       0.73      0.62      0.67       160
           2       0.70      0.70      0.70       160
           3       0.58      0.54      0.56       120

   micro avg       0.70      0.61      0.65       640
   macro avg       0.69      0.61      0.65       640
weighted avg       0.70      0.61      0.65       640
 samples avg       0.61      0.61      0.61       640

              precision    recall  f1-score   support

           0       0.73      0.54      0.62       200
           1       0.83      0.62      0.71       160
           2       0.81      0.71      0.76       160
           3       0.65      0.52      0.58       120

   micro avg       0.76      0.60      0.67       640
   macro avg       0.76      0.60      0.67       640
weighted avg       0.76      0.60      0.67       640
 samples avg       0.58      0.60      0.58       640



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
